# Imports

In [2]:
import matplotlib.pyplot as plt
import numpy as np
import os
from os import listdir as LD, makedirs as MDs
from os.path import join as PJ, basename as PBN, dirname as PDN, exists as PE
import pandas as pd
from datetime import datetime as DT

In [3]:
from imod import msw
from imod import mf6
import primod

In [4]:
import WS_Mdl.utils as U
import WS_Mdl.utils_imod as UIM

In [5]:
import importlib as IL
IL.reload(U)
IL.reload(UIM)

<module 'WS_Mdl.utils_imod' from 'C:\\Users\\Karam014\\OneDrive - Universiteit Utrecht\\WS_Mdl\\code\\WS_Mdl\\utils_imod.py'>

In [ ]:
# Import sfrmaker and other necessary packages for SFR network creation
import sfrmaker
import geopandas as gpd
import numpy as np

In [ ]:
sfrmaker.__version__

'0.12.1'

# SFR Network Creation

Creating an SFR (Streamflow Routing) network from the Chaamse beken shapefile using sfrmaker package.

In [ ]:
# Import the shapefile
Pa_Shp = r"C:\OD\WS_Mdl\models\NBr\In\SFR\_other\rivercourse\Chaamse_beken_primary_secondary.shp"

# Read the shapefile using geopandas
gdf = gpd.read_file(Pa_Shp)

print("Shapefile loaded successfully!")
print(f"Number of features: {len(gdf)}")
print(f"CRS: {gdf.crs}")
print("\nColumn names:")
print(gdf.columns.tolist())
print("\nFirst few rows:")
gdf.head()

DataSourceError: C:\OD\WS_Mdl\models\NBr\In\SFR\_other\rivercourse\Chaamse_beken_primary_secondary.shp: No such file or directory

In [ ]:
# Examine the shapefile data in more detail
print("Data types:")
print(gdf.dtypes)
print("\nGeometry types:")
print(gdf.geometry.geom_type.value_counts())
print("\nSpatial extent:")
print(f"Bounds: {gdf.total_bounds}")

# Check for any null values
print("\nNull values per column:")
print(gdf.isnull().sum())

In [ ]:
# Create SFR network using sfrmaker
# Need to create integer IDs since sfrmaker expects integer identifiers

print("Setting up data for sfrmaker...")

# Create a copy and set CRS
gdf_copy = gdf.copy()
gdf_copy.crs = "EPSG:28992"  # RD New (Netherlands)

# Create integer IDs (sfrmaker requires integer IDs)
gdf_copy['segment_id'] = range(1, len(gdf_copy) + 1)

# Create mapping dictionaries
name_to_id = dict(zip(gdf_copy['Name'], gdf_copy['segment_id']))
source_to_name = {}
for idx, row in gdf_copy.iterrows():
    source_to_name[row['Source']] = row['Name']

# Create routing: map Target coordinates to segment IDs
gdf_copy['toid'] = gdf_copy['Target'].map(source_to_name).map(name_to_id)
gdf_copy['toid'] = gdf_copy['toid'].fillna(0)  # Fill NaN with 0 (outlet)

# Convert toid to integer
gdf_copy['toid'] = gdf_copy['toid'].astype(int)

print(f"Routing setup:")
print(f"Total segments: {len(gdf_copy)}")
print(f"Segments with outlets (toid=0): {(gdf_copy['toid'] == 0).sum()}")
print(f"Segments with connections: {(gdf_copy['toid'] != 0).sum()}")

# Now create the Lines object
try:
    lines = sfrmaker.Lines.from_dataframe(
        gdf_copy,
        id_column='segment_id',
        routing_column='toid',
        name_column='Name',  # Use original Name for reference
        geometry_column='geometry'
    )
    print("\nLines object created successfully!")
    print(f"Number of line segments: {len(lines.df)}")
    print(f"Total length: {lines.df.geometry.length.sum():.2f} meters")
    
    # Display some basic information
    print("\nLines dataframe columns:")
    print(lines.df.columns.tolist())
    print("\nFirst few rows of lines dataframe:")
    print(lines.df[['id', 'toid', 'name', 'geometry']].head())
    
except Exception as e:
    print(f"Error creating Lines object: {e}")
    lines = None

In [ ]:
# Work with the Lines object directly to create SFR network information
# Since we don't have a model grid, we'll extract the key information from the Lines object

print("Working with Lines object to extract SFR network information...")

# The Lines object contains our routing information
reach_data = lines.df.copy()

print("SFR Network Information:")
print(f"Total number of stream segments: {len(reach_data)}")
print(f"Total stream length: {reach_data.geometry.length.sum():.2f} meters")
print(f"Average segment length: {reach_data.geometry.length.mean():.2f} meters")

# Analyze connectivity
outlets = reach_data[reach_data['toid'] == 0]
connected = reach_data[reach_data['toid'] != 0]

print(f"\nNetwork connectivity:")
print(f"Number of outlet segments (toid=0): {len(outlets)}")
print(f"Number of connected segments: {len(connected)}")

print(f"\nOutlet segments:")
for idx, outlet in outlets.iterrows():
    print(f"  - {outlet['name']} (ID: {outlet['id']}) - Length: {outlet.geometry.length:.2f}m")

# Check for routing consistency
print(f"\nRouting validation:")
all_ids = set(reach_data['id'])
referenced_ids = set(reach_data['toid']) - {0}  # Remove outlet indicator
missing_targets = referenced_ids - all_ids
print(f"Missing target IDs: {len(missing_targets)}")
if missing_targets:
    print(f"Missing IDs: {list(missing_targets)[:10]}...")  # Show first 10

# Create a summary of the network structure
network_summary = {
    'total_segments': len(reach_data),
    'total_length_m': reach_data.geometry.length.sum(),
    'outlet_segments': len(outlets),
    'connected_segments': len(connected),
    'crs': str(reach_data.crs) if hasattr(reach_data, 'crs') else 'Unknown'
}

print(f"\nNetwork Summary:")
for key, value in network_summary.items():
    print(f"  {key}: {value}")

# Store the processed data for later use
sfr_lines = lines
sfr_reach_data = reach_data

print(f"\nSFR network data prepared successfully!")

In [ ]:
# Visualize the SFR network
fig, ax = plt.subplots(figsize=(12, 8))

# Plot the original lines
gdf.plot(ax=ax, color='blue', linewidth=1, alpha=0.7, label='Original lines')

# Plot the SFR reaches if available
if hasattr(sfr_network, 'reach_data') and 'geometry' in sfr_network.reach_data.columns:
    sfr_network.reach_data.plot(ax=ax, color='red', linewidth=2, alpha=0.8, label='SFR reaches')

ax.set_title('SFR Network from Chaamse Beken Shapefile')
ax.legend()
ax.set_xlabel('X coordinate')
ax.set_ylabel('Y coordinate')
plt.tight_layout()
plt.show()

# Print network connectivity information
print("\nNetwork connectivity:")
if hasattr(sfr_network, 'segment_data') and sfr_network.segment_data is not None:
    print(f"Segments with outlets: {(sfr_network.segment_data['outseg'] == 0).sum()}")
    print(f"Connected segments: {(sfr_network.segment_data['outseg'] > 0).sum()}")
else:
    print("Segment data not available")

In [ ]:
# Export SFR network data
output_dir = r"C:\OD\WS_Mdl\models\NBr\In\SFR\_other\rivercourse"

# Save reach data to shapefile
reach_output_path = PJ(output_dir, "sfr_reaches.shp")
if hasattr(sfr_network, 'reach_data'):
    # Convert reach data to GeoDataFrame if it isn't already
    if not isinstance(sfr_network.reach_data, gpd.GeoDataFrame):
        reach_gdf = gpd.GeoDataFrame(sfr_network.reach_data)
    else:
        reach_gdf = sfr_network.reach_data.copy()
    
    reach_gdf.to_file(reach_output_path)
    print(f"SFR reaches saved to: {reach_output_path}")

# Save segment data to CSV
if hasattr(sfr_network, 'segment_data') and sfr_network.segment_data is not None:
    segment_output_path = PJ(output_dir, "sfr_segments.csv")
    sfr_network.segment_data.to_csv(segment_output_path, index=False)
    print(f"SFR segment data saved to: {segment_output_path}")

# Print summary statistics
print("\n=== SFR Network Summary ===")
print(f"Total number of reaches: {len(sfr_network.reach_data) if hasattr(sfr_network, 'reach_data') else 'N/A'}")
if hasattr(sfr_network, 'segment_data') and sfr_network.segment_data is not None:
    print(f"Total number of segments: {len(sfr_network.segment_data)}")
    print(f"Number of outlet segments: {(sfr_network.segment_data['outseg'] == 0).sum()}")

print("\nSFR network creation completed successfully!")

In [ ]:
# Final Summary and Export Options
print("=== SFR Network Creation Summary ===")
print(f"✓ Successfully imported shapefile: {PBN(shapefile_path)}")
print(f"✓ Created Lines object with {len(sfr_lines.df)} segments")
print(f"✓ Total network length: {sfr_lines.df.geometry.length.sum():.2f} meters")
print(f"✓ Coordinate Reference System: {sfr_lines.df.crs}")

print("\n=== Available Data ===")
print("1. Lines object (sfr_lines) - contains routing and geometry")
print("2. Reach data (sfr_reach_data) - processed segment information")
print("3. Original shapefile data (gdf) - raw input data")

print("\n=== Next Steps for MODFLOW Integration ===")
print("To use this SFR network in MODFLOW, you will need to:")
print("1. Define a MODFLOW model grid")
print("2. Use sfr_lines.to_sfr(model) to create SFR package")
print("3. Add stream properties (width, depth, roughness, etc.)")
print("4. Define boundary conditions (inflows, diversions)")

print("\n=== Example Export Commands ===")
print("# Export to shapefile:")
print("sfr_reach_data.to_file('sfr_network.shp')")
print("")
print("# Export to CSV:")
print("sfr_reach_data.drop('geometry', axis=1).to_csv('sfr_network.csv')")
print("")
print("# Create SFR package (when you have a model):")
print("sfr_package = sfr_lines.to_sfr(model)")

# Show key columns available
print(f"\n=== Available Columns in SFR Data ===")
for col in sfr_reach_data.columns:
    if col != 'geometry':
        print(f"  {col}: {sfr_reach_data[col].dtype}")
        
print("\nSFR network is ready for integration with MODFLOW!")